In [1]:
import pandas as pd
from pathlib import Path
X_train = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\X_train.csv"),sep=r"\s+")
Y_train = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\Y_train.csv"),header=None, names=['category'])
X_test = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\X_test.csv"),sep=r"\s+")
Y_test = pd.read_csv(Path(r"C:\Users\90553\Documents\ml_data_analysis\data\Y_test.csv"),header=None, names=['category'])
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(1668, 9635) (1668, 1) (557, 9635) (557, 1)


In [2]:
XY_train = pd.concat([Y_train, X_train], axis=1)
print(XY_train.head())

   category  ad  sale  boost  time  warner  profit  quarterli  media  giant  \
0         4   1     1      0     0       0       0          0      4      1   
1         1   0     0      0     0       0       0          0      0      0   
2         1   1     0      0     0       0       0          0      0      0   
3         0   0     0      1     0       0       0          0      0      0   
4         0   0     0      0     1       0       0          0      0      0   

   ...  Â£339  denialofservic  ddo  seagrav  bot  wirelessli  streamcast  \
0  ...      0               0    0        0    0           0           0   
1  ...      0               0    0        0    0           0           0   
2  ...      0               0    0        0    0           0           0   
3  ...      0               0    0        0    0           0           0   
4  ...      0               0    0        0    0           0           0   

   peripher  headphon  flavour  
0         0         0        0  
1 

In [3]:
word_freq_sum = XY_train.groupby('category').sum()
word_freq_sum = word_freq_sum + 1  # Dirichlet prior alpha=1
print(word_freq_sum.head())



           ad  sale  boost  time  warner  profit  quarterli  media  giant  \
category                                                                    
0         126   285     88   165       1     188         10     24     70   
1          89    96     13   160      11       8          1     32     13   
2         143     9     35   240       1       4          2     28      4   
3         158    35     17   263       1       1          1     20      6   
4         120    92     18   285      11      19          3    146     46   

          jump  ...  Â£339  denialofservic  ddo  seagrav  bot  wirelessli  \
category        ...                                                         
0           29  ...      1               1    1        1    1           1   
1            5  ...      1               1    1        1    1           1   
2            4  ...      1               1    1        1    1           1   
3           41  ...      1               1    1        1    1           1  

In [4]:
total_words_per_category = word_freq_sum.sum(axis=1)
print(total_words_per_category)

category
0    73524
1    57645
2    81189
3    72074
4    88068
dtype: int64


In [5]:
import numpy as np
word_probs = word_freq_sum.div(total_words_per_category, axis=0)

# take log normally (this will give -inf where prob=0)
log_probs = np.log(word_probs)

# replace -inf with a very negative number to simulate it
#log_probs = log_probs.replace(-np.inf, -1e12)


print(log_probs.head())
print(log_probs.shape)
print(word_probs.sum(axis=1))

                ad      sale     boost      time     warner     profit  \
category                                                                 
0        -6.369085 -5.552878 -6.728030 -6.099422 -11.205367  -5.968925   
1        -6.473422 -6.397711 -8.397109 -5.886885  -8.564164  -8.882617   
2        -6.341690 -9.107310 -7.749187 -5.823896 -11.304535  -9.918241   
3        -6.122854 -7.630101 -8.352235 -5.613295 -11.185449 -11.185449   
4        -6.598373 -6.864076 -8.495493 -5.733375  -8.987969  -8.441426   

          quarterli     media     giant      jump  ...      Â£339  \
category                                           ...              
0         -8.902782 -8.027313 -6.956872 -7.838071  ... -11.205367   
1        -10.962059 -7.496323 -8.397109 -9.352621  ... -10.962059   
2        -10.611388 -7.972331 -9.918241 -9.918241  ... -11.304535   
3        -11.185449 -8.189716 -9.393689 -7.471877  ... -11.185449   
4        -10.287252 -6.402258 -7.557223 -9.594105  ... -10.287252  

In [6]:
X = X_test.to_numpy() #mxn matrix of test data
T = log_probs.to_numpy() #kxn matrix of log prob of words given class
print(X.shape)
print(T.shape)


(557, 9635)
(5, 9635)


In [7]:
scores = X @ T.T              # (n_docs × n_classes)

print(scores.shape)           # should be (num_docs, num_categories)
print(scores)

(557, 5)
[[-1174.53356406  -955.81376786 -1124.47787562 -1167.36945432
  -1092.77343317]
 [-1182.67179223 -1055.99295045 -1207.18557212 -1214.87869047
  -1168.74429205]
 [-1441.62986423 -1722.73462281 -1710.10251327 -1812.0779455
  -1679.44346606]
 ...
 [-1938.87913634 -1939.78444594 -2011.28846859 -2062.46344231
  -1802.8878902 ]
 [-1536.91451487 -1672.52815719 -1631.47496219 -1692.66721965
  -1656.75548517]
 [-2576.58849266 -2547.4542557  -2529.44395554 -2246.86668147
  -2580.6231312 ]]


In [8]:
priors = Y_train['category'].value_counts().sort_index() / len(Y_train)
print(priors)
print(priors.sum())
priors = np.log(priors)

priors = priors.reset_index()
priors.columns = ['category', 'probability']
priors_array = priors['probability'].to_numpy()
print(priors_array)

category
0    0.224820
1    0.170264
2    0.191247
3    0.226019
4    0.187650
Name: count, dtype: float64
1.0
[-1.49245456 -1.77040634 -1.65418948 -1.4871354  -1.67317739]


In [9]:
final_scores = scores + priors_array
print(final_scores.shape)
print(final_scores)


(557, 5)
[[-1176.02601862  -957.5841742  -1126.1320651  -1168.85658971
  -1094.44661057]
 [-1184.16424679 -1057.7633568  -1208.8397616  -1216.36582586
  -1170.41746945]
 [-1443.12231878 -1724.50502915 -1711.75670275 -1813.56508089
  -1681.11664345]
 ...
 [-1940.3715909  -1941.55485228 -2012.94265807 -2063.9505777
  -1804.56106759]
 [-1538.40696943 -1674.29856354 -1633.12915167 -1694.15435505
  -1658.42866256]
 [-2578.08094722 -2549.22466204 -2531.09814502 -2248.35381687
  -2582.29630859]]


In [12]:
import numpy as np



# 1. Predicted class indices
# np.argmax already picks the smallest index when values are equal
y_pred = np.argmax(final_scores, axis=1)

# 2. True labels
y_true = Y_test.to_numpy().flatten()   # or np.array(Y_test).flatten() if already numeric

# 3. Accuracy
accuracy = np.mean(y_true == y_pred)
print(f"Accuracy: {accuracy:.3f}")

# 4. Confusion matrix (NumPy only)
K = len(np.unique(y_true))
conf_matrix = np.zeros((K, K), dtype=int)
for t, p in zip(y_true, y_pred):
    conf_matrix[t, p] += 1

print("Confusion matrix")
print(conf_matrix)



Accuracy: 0.977
Confusion matrix
[[131   0   2   0   2]
 [  0  97   0   0   5]
 [  1   0  96   0   1]
 [  0   0   1 133   0]
 [  1   0   0   0  87]]
